##### IMPORT

In [34]:
import os
import json
import tqdm 
import pandas as pd
from operator import itemgetter
from IPython.display import display, HTML, Markdown

from langchain_groq import ChatGroq
from langchain_cerebras import ChatCerebras

from langchain_openai import AzureChatOpenAI
from langchain_community.vectorstores import Chroma, FAISS
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader, PDFPlumberLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    AIMessage,
    trim_messages
)
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from dotenv import load_dotenv
load_dotenv()
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['CEREBRAS_API_KEY'] = os.getenv("CEREBRAS_API_KEY")
os.environ['ENDPOINT_URL'] = os.getenv("ENDPOINT_URL")
os.environ['AZURE_OPENAI_API_KEY'] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ['AZURE_OPENAI_API_VERSION'] = os.getenv("AZURE_OPENAI_API_VERSION")

In [35]:
sample_questions = [
    "Current revenue model - Fixed Fee/Time & Material Contract/Volume based Invoicing (Please select multiple option if required)",
    "Are we required to share FTE details at the time of invoicing?",
    "Does GEP have a negotiated rate card with the client?",
    "Have we built any year-over-year efficiencies into the solution/SOW? If yes, what are the committed efficiency targets?",
    "What are the start and end dates of this SOW",
    "what is the renewal clause",
    "Is there Termination for convenience",
    "Is travel billable or not",
    "what are the major SLA's",
    "is there clause for fees at risk",
    "What are the payment terms",
    "is there Clause for COLA",
    "what is the category of work we are doing",
    "What is the credit period of the contract",
    "what is the invoicing schedule"
]

##### INGESTION - SPLITTING - EMBEDDINGS - PROMPT

In [ ]:
# intfloat/e5-large-v2 | BAAI/bge-base-en-v1.5 | Qwen/Qwen3-Embedding-0.6B
# embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5") 
# vectordb=Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
model_name = "Qwen/Qwen3-Embedding-0.6B"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [37]:
BASE_DIR='./FW__Contracts'
doc_list=os.listdir(BASE_DIR)
doc0_name=doc_list[0]
doc0=PDFPlumberLoader(BASE_DIR+'/'+doc0_name).load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
f_docs=text_splitter.split_documents(doc0)

Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value


In [38]:
vectordb=Chroma.from_documents(documents=f_docs,embedding=hf, persist_directory="./chroma_db")

KeyboardInterrupt: 

In [ ]:
# prompt = ChatPromptTemplate.from_template(
#     """
#     Answer the following question strictly based on the provided context. Keep the response highly brief, short, and crisp.:
#     <context>
#     {context}
#     </context>
#     """)

# #* OUTPUT FORMAT in PD DataFrame
# results = []
# for i in tqdm.tqdm(range(len(sample_questions))):
#     query = sample_questions[i]
#     response = retrieval_chain.invoke({"input": query})
#     results.append([query, response['answer']])
# df = pd.DataFrame(results, columns=["Question", "Answer"])
# with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
#     display(df.style.set_properties(**{'text-align': 'left'}).set_table_styles(
#         [{'selector': 'th', 'props': [('text-align', 'left')]}]
#     ))

#############################################################################################################################################################
# Gemma2-9b-It
llm = ChatGroq(model="openai/gpt-oss-120b", groq_api_key=os.environ['GROQ_API_KEY'])
# llm = ChatCerebras(model="gpt-oss-120b", cerebras_api_key=os.environ['CEREBRAS_API_KEY'])

prompt = ChatPromptTemplate.from_messages([
    ("system",
     """Answer the following question strictly based on the provided context.
     Keep the response highly brief, short, and crisp.:
     <context>
     {context}
     </context>"""
    ),
    MessagesPlaceholder(variable_name="messages")
])

retrieval_chain = create_retrieval_chain(vectordb.as_retriever(), create_stuff_documents_chain(llm, prompt))
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | retrieval_chain
    | (lambda x: {"output": x["answer"], **x})
)

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable": {"session_id": "test_rag1"}}
query = "which are the 2 parties between which the contract is signed?"
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content=query)],
        "input": query,
    },
    config=config,
)

In [ ]:
llm = ChatCerebras(
    model="gpt-oss-120b",  # | "llama3.1-8b",
    cerebras_api_key=os.environ['CEREBRAS_API_KEY']
)

# Your existing prompt and retrieval chain
prompt = ChatPromptTemplate.from_messages([
    ("system",
     """Answer the following question strictly based on the provided context.
     Keep the response highly brief, short, and crisp.:
     <context>
     {context}
     </context>"""
    ),
    MessagesPlaceholder(variable_name="messages")
])

retrieval_chain = create_retrieval_chain(
    vectordb.as_retriever(), 
    create_stuff_documents_chain(llm, prompt)
)

# Memory store
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | retrieval_chain
    | (lambda x: {"output": x["answer"], **x})
)

# Create chain with memory
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

# Usage
config = {"configurable": {"session_id": "test_rag1"}}
query = "which are the 2 parties between which the contract is signed?"

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content=query)],
        "input": query,
    },
    config=config,
)

print(response["answer"])

The contract is between WestRock and the Supplier.


##### EVALUATION

In [33]:
results = []
config = {"configurable": {"session_id": "batch_test"}}

for query in tqdm.tqdm(sample_questions):
    response = with_message_history.invoke(
        {
            "messages": [HumanMessage(content=query)],
            "input": query,
        },
        config=config,
    )
    results.append([query, response["output"]])  
    
df = pd.DataFrame(results, columns=["Question", "Answer"])
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(
        df.style.set_properties(**{'text-align': 'left'})
        .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
    )

100%|██████████| 15/15 [00:11<00:00,  1.36it/s]


,Question,Answer
0,Current revenue model - Fixed Fee/Time & Material Contract/Volume based Invoicing (Please select multiple option if required),- Fixed Fee - Volume‑based Invoicing
1,Are we required to share FTE details at the time of invoicing?,No. The provided documents do not require sharing FTE details when invoicing.
2,Does GEP have a negotiated rate card with the client?,The provided documents do not mention a negotiated rate card between GEP and the client.
3,"Have we built any year-over-year efficiencies into the solution/SOW? If yes, what are the committed efficiency targets?","The excerpt does not specify any year‑over‑year efficiency targets. It describes the types of savings (soft, tangible, working‑capital) and the activities needed to realize them, but it does not include concrete YoY efficiency commitments."
4,What are the start and end dates of this SOW,**Start date:** May 15 2023 **End date:** February 28 2024
5,what is the renewal clause,The provided excerpt does not contain any renewal clause.
6,Is there Termination for convenience,"Yes. The document refers to termination “for convenience” (e.g., “if the Supplement were terminated for convenience…”)."
7,Is travel billable or not,The provided context does not contain any information about travel being billable or non‑billable.
8,what are the major SLA's,"The key Service Level (SLA) categories in the document are: | Type | Workstream | Service‑Level Title | |------|-----------|----------------------| | **CSL 1** | S2C | Cumulative Realized Hard Savings | | **CSL 2** | P2P | PO Creation Timeliness | | **CSL 3** | S2P | WestRock Satisfaction Survey | | **CSL 4** | P2P | Helpdesk Ticket Resolution | | **CSL 5** | Technology | System Uptime | In addition, the contract defines ten KPIs (KPI 1‑10) that are also measured, covering procurement audit compliance, spend management, vendor diversity, contract spend, spot‑buy efficiency, PO acknowledgment, soft savings, catalog pricing completeness, and blocked‑invoice percentage."
9,is there clause for fees at risk,"Yes. Clause 2.27 defines “Fees at Risk,” referring to its meaning in Schedule A‑3, and Schedule A‑3 outlines true‑up procedures for Variable Fees and Fees at Risk."
